<a href="https://colab.research.google.com/github/saadbelefqih/niod/blob/main/Neural_Interpretable_Outlier_Detection_(NIOD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Input,Lambda
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import BaggingClassifier
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from tensorflow.keras.losses import MeanSquaredError
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import plot_tree
import networkx as nx
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from scipy.spatial.distance import cdist
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, average_precision_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, accuracy_score, precision_score, recall_score, f1_score
from typing import Union, Tuple
import torch.nn.functional as F
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.spatial.distance import cdist
import numpy as np
from typing import List, Tuple
import torch
import torch.nn as nn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
url = "/content/drive/MyDrive/OutlierDetectionDiabetes/diabetes.csv"
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
# Load your dataset (replace 'your_data.csv' with your actual file)
data = pd.read_csv(url)
# Handle missing values (replace 'method' with your preferred imputation method)
data = data.fillna(data.mean())
# Normalize features
#scaler = StandardScaler()
#data[['glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi', 'age']] = scaler.fit_transform(data[['glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi', 'age']])
# Split the data into features and target variable
X = data.drop('Outcome', axis=1)
#y = data['outcome']  # For evaluation purposes only
print("X:")
print(X.head())

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Concatenate
import tensorflow_probability as tfp
from sklearn.metrics import roc_curve, auc, classification_report,accuracy_score, precision_score, recall_score, f1_score
import matplotlib.gridspec as gridspec

class CentroidDistanceLayer(tf.keras.layers.Layer):
    def __init__(self, input_shape, **kwargs):
        super(CentroidDistanceLayer, self).__init__(**kwargs)
        # Initialize centroid with a larger initial value
        self.input_shape = input_shape
        self.initial_centroid = [0.21565644, 0.15952514, -0.16965939, 0.24391688]
        # Initialize centroid as a trainable variable with a larger initial value
        self.centroid = self.add_weight(
            shape=(input_shape,), # Shape based on input features
            initializer=tf.keras.initializers.Constant(self.initial_centroid),  # Your initial value
            trainable=True,  # Make it trainable
            name='centroid'
        )

        # Store initial values as non-trainable weights for reference
        self.initial_centroid_const = self.add_weight(
            shape=(input_shape,),
            initializer=tf.keras.initializers.Constant(self.initial_centroid),
            trainable=False,
            name='initial_centroid'
        )

    def call(self, inputs):

        if self.centroid is None:
            raise ValueError("Centroid has not been initialized. Call set_centroid() first.")
        # Ensure inputs and centroid have the same dtype
        inputs = tf.cast(inputs, dtype=self.centroid.dtype)
        # Calculate distance without expanding dimensions
        distance = tf.reduce_sum(tf.square(inputs - self.centroid), axis=-1)
        return tf.expand_dims(distance, -1)  # Expand dims for binary classification


    """def set_centroid(self, new_centroid):
        self.centroid = tf.convert_to_tensor(new_centroid, dtype=tf.float32)"""

    def get_centroid(self):
        return self.centroid.numpy() if self.centroid is not None else None


class BinaryClassificationLayer(tf.keras.layers.Layer):
    def __init__(self, threshold=0.11299490928649902, **kwargs):
        super(BinaryClassificationLayer, self).__init__(**kwargs)
        self.threshold = self.add_weight(
            name='classification_threshold',
            shape=(),
            initializer=tf.keras.initializers.Constant(threshold),
            trainable=True
        )

    def call(self, inputs):
        # Get probability scores from input
        distances = inputs

        # Apply threshold for binary classification
        binary_output = tf.cast(distances >= self.threshold, tf.float32)
        return binary_output

    def set_threshold(self, new_threshold):
        self.threshold.assign(new_threshold)

    def get_threshold(self):
        return self.threshold.numpy()


class HybridNeuralMultiTreeModel:
    def __init__(self, input_shape, n_components=10, embedding_dim=8, n_centroids=2,
                 tree_max_depth=4, n_trees=2, distance_percentile=75, training=False, seed=42):
        self.input_shape = input_shape
        self.n_components = n_components
        self.embedding_dim = embedding_dim
        self.n_centroids = n_centroids
        self.tree_max_depth = tree_max_depth
        self.n_trees = n_trees
        self.distance_percentile = distance_percentile
        self.scaler = StandardScaler()
        self.seed = seed
        self.training = training
        # Set random seed for reproducibility
        tf.random.set_seed(seed)

        # If not training, set dropout rate to 0 to make it deterministic
        self.dropout_rate = 0.2 if training else 0.0
        # Build the main model
        self.model, self.dim_reduction_model, self.embedding_model = self.build_model()

    def build_model(self):
        inputs = Input(shape=self.input_shape, name="input_layer")

        # Dimensionality reduction layers
        x = Dense(self.n_components, activation='relu', name='dim_reduction_1',kernel_initializer=tf.keras.initializers.GlorotUniform(seed=self.seed))(inputs)
        x = Dropout(self.dropout_rate)(x, training=self.training)
        x = Dense(self.n_components // 2, activation='relu', name='dim_reduction_2',kernel_initializer=tf.keras.initializers.GlorotUniform(seed=self.seed))(x)

        # Embedding layers
        e = Dense(self.embedding_dim, activation='tanh',name='embedding_1',kernel_initializer=tf.keras.initializers.GlorotUniform(seed=self.seed))(x)
        e = Dropout(self.dropout_rate)(e, training=self.training)
        e = Dense(self.embedding_dim // 2, activation='tanh',name='embedding_2',kernel_initializer=tf.keras.initializers.GlorotUniform(seed=self.seed))(e)

        # CentroidDistance layers
        self.centroid_layer = CentroidDistanceLayer(input_shape=e.shape[-1])
        outlier_score = self.centroid_layer(e)

        # Binary Classification Layer
        self.classificationLayer = BinaryClassificationLayer()
        output = self.classificationLayer(outlier_score)  # Get the output tensor


        # Create models
        main_model = Model(inputs=inputs, outputs=output, name="Hybrid_Neural_Tree")

        @tf.function
        def custom_loss(y_true, y_pred):
            # Label smoothing
            label_smoothing = 0.1
            y_true = y_true * (1 - label_smoothing) + 0.5 * label_smoothing

            # Binary cross-entropy
            bce = tf.keras.losses.binary_crossentropy(
                y_true,
                y_pred,
                from_logits=False
            )

            # Weights for different loss components
            centroid_preservation_weight = 5.0  # Increased weight for centroid preservation
            class_balance_weight = 0.5
            centroid_regularization_weight = 0.1

            # Centroid preservation loss (penalize deviation from initial values)
            centroid_preservation_loss = tf.reduce_mean(
                tf.square(self.centroid_layer.centroid - self.centroid_layer.initial_centroid_const)
            )

            # Original regularization terms
            centroid_mean = tf.reduce_mean(self.centroid_layer.centroid)
            centroid_regularization = tf.reduce_sum(tf.square(self.centroid_layer.centroid - centroid_mean))
            class_balance = class_balance_weight * tf.square(tf.reduce_mean(y_pred) - 0.5)

            # Combined loss with strong weight on centroid preservation
            total_loss = (bce +
                         centroid_preservation_weight * centroid_preservation_loss +
                         centroid_regularization_weight * centroid_regularization +
                         class_balance)

            return total_loss

        main_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
            loss=custom_loss,
            metrics=['accuracy']
        )

        dim_reduction_model = Model(inputs=inputs, outputs=x, name="Dimensionality_Reduction_Model")
        embedding_model = Model(inputs=x, outputs=e, name="Embedding_Model")

        return main_model, dim_reduction_model, embedding_model

    def fit(self, X, epochs=200, batch_size=32, validation_split=0.2):
        #scaler = StandardScaler()
        #X_scaled = scaler.fit_transform(X)
        X_scaled = X

        # Get initial embeddings
        reduced_features = self.dim_reduction_model.predict(X_scaled)
        embedding_features = self.embedding_model.predict(reduced_features)

        centroid = np.mean(embedding_features, axis=0)
        # Calculate distances of each point from the centroid in latent space
        #distances = cdist(embedding_features, centroid.reshape(1, -1), metric='euclidean').flatten()
        distances = tf.reduce_sum(tf.square(embedding_features - centroid), axis=-1)
        distances=tf.expand_dims(distances, -1)
        # Set a threshold for outlier detection based on the 95th percentile of distances
        threshold = np.percentile(distances, 75)
        # Initialize centroids with normalized data
        tf.print("fit centroid:", centroid)
        tf.print("fit threshold:", threshold)

        #self.centroid_layer.set_weights([centroid])
        #self.classificationLayer.set_weights([np.array(threshold)])
        # Normalize centroids
        #initial_centroids = initial_centroids / np.linalg.norm(initial_centroids, axis=1, keepdims=True)
        #self.centroid_layer.set_centroid(centroid)

        # Calculate adaptive threshold
        #self.classificationLayer.set_threshold(threshold)

        # Generate balanced binary labels
        #binary_labels = (distances > threshold).astype(int)
        #binary_labels = binary_labels.reshape(-1, 1)
        binary_labels = tf.cast(distances >= threshold, tf.float32).numpy()

        # Training callbacks
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=15,  # Increased patience
                restore_best_weights=True
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.2,  # More aggressive LR reduction
                patience=10,
                min_lr=0.00001
            ),
            # Add callback to monitor centroid values
            tf.keras.callbacks.LambdaCallback(
                on_epoch_end=lambda epoch, logs: tf.print(
                    "\nCurrent centroids:", self.centroid_layer.centroid,
                    "\nTarget centroids:", self.centroid_layer.initial_centroid_const,
                    "\nDifference:", tf.reduce_mean(tf.abs(
                        self.centroid_layer.centroid - self.centroid_layer.initial_centroid_const
                    ))
                ) if epoch % 10 == 0 else None
            )
        ]

        history = self.model.fit(
            X_scaled, binary_labels,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=validation_split,
            callbacks=callbacks,
            verbose=1
        )

        # Train tree ensemble
        self.tree_classifier = BaggingClassifier(
            estimator=DecisionTreeClassifier(max_depth=self.tree_max_depth),
            n_estimators=self.n_trees,
            random_state=42
        )
        self.tree_classifier.fit(X_scaled, binary_labels.ravel())

        return history

    def predict(self, X):
        return self.model.predict(X)

    def get_threshold(self):
        return self.classificationLayer.get_threshold()

    def get_centroid(self):
        return self.centroid_layer.get_centroid()

    def evaluate_performance(self, X_test, y_true):
        """
        Evaluates the model's performance on test data and plots accuracy, precision, recall, F1-score, and confusion matrix.

        Parameters:
        X_test (ndarray): Test feature data.
        y_true (ndarray): True binary labels for the test data.
        """
        # Obtain binary predictions from centroid-based model
        y_true = y_true.astype(int).ravel()
        prediction = self.predict(X_test)
        print("evaluate_performance = prediction")
        #print(prediction)
        print("************************")
        # Calculate overall  metrics
        acc = accuracy_score(y_true, prediction)
        prec = precision_score(y_true, prediction)
        rec= recall_score(y_true, prediction)
        f1 = f1_score(y_true, prediction)
        # Calculate per-class metrics using classification_report
        report = classification_report(y_true, prediction, target_names=['Normal', 'Outlier'], output_dict=True)
        # 1. Metrics Bar Plot
        plt.figure(figsize=(8, 6))
        metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
        values = [acc, prec, rec, f1]
        colors = ['#2ecc71', '#3498db', '#e74c3c', '#f1c40f']

        plt.bar(metrics, values, color=colors)
        plt.ylim(0, 1)
        plt.title('Model Performance Metrics', pad=20)
        plt.grid(True, alpha=0.3)

        # Add value labels on bars
        for bar in plt.gca().patches:
            height = bar.get_height()
            plt.gca().text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom')
        plt.show()

        # 2. Per-Class Metrics Plot
        # Create a pandas DataFrame for better visualization
        df_report = pd.DataFrame(report).transpose()

        print("Per-Class Metrics:")
        print(df_report)
        # 2. Confusion Matrix
        fig, ax = plt.subplots(figsize=(6, 6))
        cm = confusion_matrix(y_true, prediction)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.show()

        # 3. ROC Curve
        fig3,ax3 = plt.subplots(figsize=(6, 6))
        # Get the distance scores (before thresholding)
        reduced_features = self.dim_reduction_model.predict(X_test)
        embedding_features = self.embedding_model.predict(reduced_features)
        distances = self.centroid_layer(embedding_features).numpy()

        # Calculate ROC curve and AUC
        fpr, tpr, _ = roc_curve(y_true, distances)
        roc_auc = auc(fpr, tpr)

        # Plot ROC curve
        ax3.plot(fpr, tpr, color='darkorange', lw=2,
                label=f'ROC curve (AUC = {roc_auc:.3f})')
        ax3.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        ax3.set_xlim([0.0, 1.0])
        ax3.set_ylim([0.0, 1.05])
        ax3.set_xlabel('False Positive Rate')
        ax3.set_ylabel('True Positive Rate')
        ax3.set_title('Receiver Operating Characteristic')
        ax3.legend(loc="lower right")
        ax3.grid(True, alpha=0.3)

        plt.show()
        # 4. Class Distribution
        plt.figure(figsize=(6, 6))
        class_dist = np.bincount(y_true.astype(int)) / len(y_true) * 100
        plt.pie(class_dist, labels=['Normal', 'Outlier'], autopct='%1.1f%%',
                colors=['#3498db', '#e74c3c'])
        plt.title('Class Distribution')

        plt.tight_layout()
        plt.show()

        # 2. Feature Importance Analysis
        plt.figure(figsize=(10, 6))
        # Get feature importance from tree ensemble
        importances = np.mean([tree.feature_importances_ for tree in self.tree_classifier.estimators_], axis=0)
        feature_importance_df = pd.DataFrame({
            'feature': [f'Feature {i}' for i in range(X_test.shape[1])],
            'importance': importances
        }).sort_values('importance', ascending=True)

        plt.barh(range(len(feature_importance_df)), feature_importance_df['importance'])
        plt.yticks(range(len(feature_importance_df)), feature_importance_df['feature'])
        plt.xlabel('Feature Importance')
        plt.title('Feature Importance Analysis')
        plt.tight_layout()
        plt.show()

        # 3. Distance Distribution Analysis
        reduced_features = self.dim_reduction_model.predict(X_test)
        embedding_features = self.embedding_model.predict(reduced_features)
        distances = self.centroid_layer(embedding_features).numpy()

        plt.figure(figsize=(10, 6))
        plt.hist(distances[y_true == 0], bins=30, alpha=0.5, label='Normal', color='blue')
        plt.hist(distances[y_true == 1], bins=30, alpha=0.5, label='Outlier', color='red')
        plt.axvline(self.get_threshold(), color='green', linestyle='--', label='Decision Threshold')
        plt.xlabel('Distance from Centroid')
        plt.ylabel('Count')
        plt.title('Distance Distribution Analysis')
        plt.legend()
        plt.show()
        # 4. Embedding Space Visualization (if embedding_dim <= 2)
        if embedding_features.shape[1] <= 2:
            plt.figure(figsize=(10, 6))
            plt.scatter(embedding_features[y_true == 0, 0],
                      embedding_features[y_true == 0, 1] if embedding_features.shape[1] > 1 else np.zeros_like(embedding_features[y_true == 0]),
                      c='blue', label='Normal', alpha=0.6)
            plt.scatter(embedding_features[y_true == 1, 0],
                      embedding_features[y_true == 1, 1] if embedding_features.shape[1] > 1 else np.zeros_like(embedding_features[y_true == 1]),
                      c='red', label='Outlier', alpha=0.6)
            centroid = self.get_centroid()
            plt.scatter(centroid[0], centroid[1] if len(centroid) > 1 else 0,
                      c='green', marker='*', s=200, label='Centroid')
            plt.title('Embedding Space Visualization')
            plt.legend()
            plt.show()
        # 5. Local Interpretation Analysis
        def get_local_importance(instance, n_samples=1000):
            # Create perturbed samples around the instance
            perturbed = np.tile(instance, (n_samples, 1))
            noise = np.random.normal(0, 0.1, perturbed.shape)
            perturbed = perturbed + noise

            # Get predictions for perturbed samples
            predictions = self.predict(perturbed)

            # Calculate feature importance based on correlation with predictions
            local_importance = np.zeros(instance.shape[0])
            for i in range(instance.shape[0]):
                local_importance[i] = np.abs(np.corrcoef(perturbed[:, i], predictions.ravel())[0, 1])

            return local_importance

        # Select a few instances for local interpretation
        n_examples = 3
        example_indices = np.random.choice(len(X_test), n_examples)

        plt.figure(figsize=(15, 5 * n_examples))
        for i, idx in enumerate(example_indices):
            local_importance = get_local_importance(X_test[idx])

            plt.subplot(n_examples, 1, i + 1)
            plt.barh(range(len(local_importance)), local_importance)
            plt.yticks(range(len(local_importance)), [f'Feature {j}' for j in range(len(local_importance))])
            plt.xlabel('Local Feature Importance')
            plt.title(f'Local Interpretation for Instance {idx} (True Label: {y_true[idx]}, Predicted: {prediction[idx]})')

        plt.tight_layout()
        plt.show()

        # Print additional interpretability metrics
        print("\nInterpretability Metrics:")
        print("-" * 50)

        # Calculate decision stability
        noise = np.random.normal(0, 0.1, X_test.shape)
        noisy_predictions = self.predict(X_test + noise)
        stability = np.mean(prediction == noisy_predictions)
        print(f"Decision Stability: {stability:.3f}")

        # Calculate average distance to decision boundary
        threshold = self.get_threshold()
        avg_distance_to_boundary = np.mean(np.abs(distances - threshold))
        print(f"Average Distance to Decision Boundary: {avg_distance_to_boundary:.3f}")

        # Feature importance summary
        print("\nTop 5 Most Important Features:")
        top_features = feature_importance_df.iloc[-5:].iloc[::-1]
        for idx, row in top_features.iterrows():
            print(f"{row['feature']}: {row['importance']:.3f}")


    def visualize_decision_trees(model, feature_names=None, class_names=['Normal', 'Outlier'], max_trees=4):
        """
        Visualize the decision trees from the BaggingClassifier ensemble

        Parameters:
        -----------
        model : HybridNeuralMultiTreeModel
            The trained hybrid model containing the tree ensemble
        feature_names : list, optional
            List of feature names for the input data
        class_names : list, optional
            Names for the classes (default: ['Normal', 'Outlier'])
        max_trees : int, optional
            Maximum number of trees to display (default: 4)
        """

        # Get the trees from the bagging classifier
        trees = model.tree_classifier.estimators_
        n_trees = min(len(trees), max_trees)

        # Calculate the grid layout
        n_cols = min(2, n_trees)
        n_rows = math.ceil(n_trees / n_cols)

        # Create figure with subplots
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15 * n_cols, 10 * n_rows))
        if n_trees == 1:
            axes = np.array([axes])
        axes = axes.ravel()  # Flatten axes array for easier indexing

        # If feature names not provided, create generic ones
        if feature_names is None:
            feature_names = [f'Feature {i}' for i in range(model.input_shape)]

        # Plot each tree
        for idx, tree in enumerate(trees[:max_trees]):
            # Plot the decision tree
            plot_tree(
                tree,
                feature_names=feature_names,
                class_names=class_names,
                filled=True,
                rounded=True,
                ax=axes[idx],
                proportion=True,
                precision=2
            )
            axes[idx].set_title(f'Decision Tree {idx + 1}')

        # Remove empty subplots if any
        for idx in range(n_trees, len(axes)):
            fig.delaxes(axes[idx])

        plt.tight_layout()

        # Add information about the model
        info_text = (
            f"Model Information:\n"
            f"Number of trees in ensemble: {len(trees)}\n"
            f"Max tree depth: {model.tree_max_depth}\n"
            f"Input shape: {model.input_shape}\n"
            f"Embedding dimension: {model.embedding_dim}"
        )

        plt.figtext(0.02, 0.02, info_text, fontsize=10, bbox=dict(facecolor='white', alpha=0.8))

        return fig

    def plot_feature_importance(model, feature_names=None):
        """
        Plot feature importance across all trees in the ensemble

        Parameters:
        -----------
        model : HybridNeuralMultiTreeModel
            The trained hybrid model containing the tree ensemble
        feature_names : list, optional
            List of feature names for the input data
        """

        if feature_names is None:
            feature_names = [f'Feature {i}' for i in range(model.input_shape)]

        # Calculate mean feature importance across all trees
        importances = np.mean([
            tree.feature_importances_
            for tree in model.tree_classifier.estimators_
        ], axis=0)

        # Sort features by importance
        indices = np.argsort(importances)[::-1]

        # Create figure
        plt.figure(figsize=(12, 6))
        plt.title('Feature Importance Across Decision Trees')
        plt.bar(range(len(importances)), importances[indices])
        plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45, ha='right')
        plt.tight_layout()

        return plt.gcf()

In [ ]:
#X_train, X_test, y_train, y_test
# Initialize model
model = HybridNeuralMultiTreeModel(
    input_shape=(X_train.shape[1],),
    n_centroids=2,
    tree_max_depth=5,
    n_trees=3
)

# Train model
history = model.fit(X_train)

# Get the threshold value
threshold = model.get_threshold()
print(f"Outlier detection threshold: {threshold}")

centroids = model.get_centroid()
print(f"Outlier detection centroids: {centroids}")

In [ ]:
model.evaluate_performance(X_test, y_test)